In [6]:
# Usando a versão 3.10.11 do python

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pathlib import Path
from datetime import timedelta
import os
pd.options.mode.chained_assignment = None

# Caminho da base
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

print(f'diretorio: {diretório_princapl}')
print(f'caminho: {caminho}')

diretorio: c:\Users\Arq. Andrea\Documents\Yuri\git\previsao-ibovespa
caminho: c:\Users\Arq. Andrea\Documents\Yuri\git\previsao-ibovespa\Base\Base Ibovespa Tratada.csv


In [7]:
acertos = 0
idx = 1

# Lista para armazenar os resultados
todos_resultados = []

while idx <= 30:

    # Carrega a base
    acao = pd.read_csv(caminho)
    acao['Data'] = pd.to_datetime(acao['Data'])

    # Ordena por data
    acao = acao.sort_values(by="Data")

    # Define data de ontem e anteontem
    ultimo_dia = acao['Data'].max()
    penultimo_dia = acao['Data'].iloc[-2]

    # print(f'Ontem: {ultimo_dia}\nAnteontem: {penultimo_dia}')

    # Filtrando base que vai ser usada para treino até o penultimo dia da base
    acao_treino = acao#[acao['Data'] < ultimo_dia]
    cotacao_treino = acao_treino['Fechamento'].to_numpy().reshape(-1, 1)

    # Armazenar tamanho dos dados de treinamento
    tamanho_dados_treinamento = int(len(cotacao_treino) * 1)

    # Escala os dados
    escalador = MinMaxScaler(feature_range=(0, 1))
    dados_escalados = escalador.fit_transform(cotacao_treino)

    # Cria dados de entrada para o modelo
    treinamento_x = []
    treinamento_y = []

    for i in range(60, len(dados_escalados)):
        treinamento_x.append(dados_escalados[i-60:i, 0])
        treinamento_y.append(dados_escalados[i, 0])

    treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
    treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

    # Define e treina o modelo
    modelo = Sequential()
    modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
    modelo.add(Dropout(0.2))
    modelo.add(LSTM(50, return_sequences=False))
    modelo.add(Dropout(0.2))
    modelo.add(Dense(25))
    modelo.add(Dense(1))
    modelo.compile(optimizer="adam", loss="mean_squared_error")
    modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=20)

    # Pega os últimos 60 dias até ANTEONTEM para previsão
    ultimos_60 = dados_escalados[-60:]
    entrada_predicao = ultimos_60.reshape(1, 60, 1)

    predicao = modelo.predict(entrada_predicao)
    predicao = escalador.inverse_transform(predicao)

    # print(f'Valor previsto: {predicao[0][0]:.2f}')

    print(f'\nTeste {idx}')
    # ---- Descomentar as quatro linhas de baixo caso for fazer teste do valor previsto do ultimo dia da base ----
    valor_real = acao[acao['Data'] == ultimo_dia]['Fechamento'].values[0]
    valor_anterior = acao[acao['Data'] == penultimo_dia]['Fechamento'].values[0]
    variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
    variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100
    # -----------------------------------------------------------------------------

    # ---- Descomentar as quatro linhas de baixo caso for prever do dia seguinte ----
    #valor_real = 137002	# Valor de fechamento do ibovespa no dia 04/06/2025 (dia seguinte do ultimo que tem na base)
    #valor_anterior = acao[acao['Data'] == ultimo_dia]['Fechamento'].values[0]
    #variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
    #variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100
    # -----------------------------------------------------------------------------

    # Direção correta?
    direcao_correta = (variacao_real * variacao_predita) > 0  # ambos com mesmo sinal
    
    # A média das diferenças absolutas entre as previsões e os valores reais. Penaliza todos os erros da mesma forma
    mae = mean_absolute_error(df_previsao['Valor Real'], df_previsao['Valor Previsto'])
    # print(f"Mean Absolute Error (MAE): {mae:.2f}")

    # Calcular o MAPE
    mape = mean_absolute_percentage_error(df_previsao['Valor Real'], df_previsao['Valor Previsto'])
    # print(f"Mean Absolute Percentage Error (MAPE): {mape:.6f} = {100*mape:.4f}%")

    # A raiz quadrada da média dos erros ao quadrado. Dá mais peso a grandes erros
    rmse = np.sqrt(mean_squared_error(df_previsao['Valor Real'], df_previsao['Valor Previsto']))
    mean_price = df_previsao['Valor Real'].mean()

    # print(f"RMSE: {rmse:.2f}")
    # print(f"RMSE percentual: {(rmse / mean_price):.6f} = {100*(rmse / mean_price):.4f}%")

    # Criar df final
    df_previsao = pd.DataFrame([{
        'Data': ultimo_dia.date(), # Data do penultimo dia da base
        #'Data Previsão': ultimo_dia.date() + timedelta(days=1), # Data do dia seguinte do ultimo dia da base
        # 'Valor Dia Anterior': f"{valor_anterior:.2f}",
        'Valor Real': f"{valor_real:.2f}",
        'Valor Previsto': f"{predicao[0][0]:.2f}",
        'Variação Real (%)': f"{variacao_real:.3f}",
        'Variação Prevista (%)': f"{variacao_predita:.3f}",
        'MAE': f"{mae:.2f}",
        'MAPE': f"{100*mape:.4f}%",
        'RMSE': f"{rmse:.2f}",
        'RMSE (%)': f"{100*(rmse / mean_price):.4f}%",
        'Direção Correta?': 'Sim' if direcao_correta else 'Não'
    }])

    # Ajustar colunas numéricas
    colunas_para_converter = ['Valor Real', 'Valor Previsto', 'Variação Real (%)', 'Variação Prevista (%)']
    for coluna in colunas_para_converter:
        df_previsao[coluna] = df_previsao[coluna].astype(float)
    
    todos_resultados.append(df_previsao)
    
    if df_previsao['Direção Correta?'].iloc[0] == 'Sim':
        acertos += 1
        print(f'total de acertos: {acertos} de 30 testes\n')

    idx +=1

# Junta todos os resultados e salva em Excel
df_final = pd.concat(todos_resultados, ignore_index=True)
df_final.to_excel("resultados_previsoes_1.xlsx", index=False)
print("\nArquivo 'resultados_previsoes_1.xlsx' salvo com sucesso!")

Epoch 1/20


c:\Users\Arq. Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


256/256 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0228
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0028
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0021
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0017
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0012
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0013
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0011
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 9.8818e-04
Epoch 9/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0011
Epoch 10/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 9.8117e-04
Epoch 11/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0011
Epoch 12/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 9.5809e-04
Epoch 13/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 9.1251e-04
Epoch 14/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0010
Epoch 15/20
256/256 ━━━━━━━━━━━━━━━━━━

c:\Users\Arq. Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 0.0265
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0035
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0023
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0017
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0015
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0012
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0012
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0010
Epoch 9/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0010
Epoch 10/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0010
Epoch 11/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0010
Epoch 12/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 9.5633e-04
Epoch 13/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 8.9093e-04
Epoch 14/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 8.9476e-04
Epoch 15/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4

...

256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.0225
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0032
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0022
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0016
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0013
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0013
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0010
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.6499e-04
Epoch 9/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.7795e-04
Epoch 10/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 9.4736e-04
Epoch 11/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.6493e-04
Epoch 12/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.6690e-04
Epoch 13/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.6018e-04
Epoch 14/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.6709e-04
Epoch 15/20
256/256 ━━━━━━

c:\Users\Arq. Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


256/256 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.0243
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0031
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0020
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0017
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0014
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0013
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.6231e-04
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.9204e-04
Epoch 9/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0011
Epoch 10/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.9853e-04
Epoch 11/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.3527e-04
Epoch 12/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0010
Epoch 13/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.3357e-04
Epoch 14/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 8.6491e-04
Epoch 15/20
256/256 ━━━━━━━━━━

c:\Users\Arq. Andrea\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


256/256 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0308
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0030
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0025
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0016
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0014
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0012
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0012
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0011
Epoch 9/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.6458e-04
Epoch 10/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.3326e-04
Epoch 11/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0011
Epoch 12/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0010
Epoch 13/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 9.6897e-04
Epoch 14/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 9.9793e-04
Epoch 15/20
256/256 ━━━━━━━━━━━━━━━━━━